In [1]:
import predict_mrcnn
import skimage.io as io
import numpy as np

import cv2

In [2]:
np.arange(3, 100, 15)

array([ 3, 18, 33, 48, 63, 78, 93])

In [ ]:
image = io.imread("/Volumes/MyPassport1/D1/depth/4973AM/_Depth_1279.png")
# image = cv2.imread("/Users/yebi/Library/CloudStorage/OneDrive-VirginiaTech/Research/Codes/research/BCS/BodyWeight/MaskRCNN/images/dataset/pic_test/4973PM_Depth_60848.png")



In [ ]:
fill_img = predict_mrcnn.predict_mrcnn(image)

In [ ]:
io.imshow(fill_img)

In [ ]:
cnts, _ = cv2.findContours(fill_img.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE) 
    # print(f"There are {len(cnts)} countorus (shapes) in total \n")
cmax = max(cnts, key=cv2.contourArea) 

In [ ]:
rect = cv2.minAreaRect(cmax)

In [ ]:
box=cv2.boxPoints(rect)
box = np.int0(box) #transfer into integers

In [ ]:
rgb_img = cv2.cvtColor(fill_img, cv2.COLOR_GRAY2RGB) 
nts_img_bi=cv2.drawContours(rgb_img.copy(), [cmax], 0, (255,0,255), 3)
area_img = cv2.drawContours(nts_img_bi.copy(), [box], 0, (0, 255, 0), 3)
# cv2.imshow("test", area_img)

In [ ]:
io.imshow(area_img)

In [ ]:
M = cv2.moments(cmax)

In [ ]:
M


In [ ]:
row_centroid = int(M["m01"] / M["m00"])
col_centroid  = int(M["m10"] / M["m00"])

In [ ]:
row_centroid, col_centroid

In [ ]:
file_path = "/Volumes/MyPassport1/D1/depth/5145PM/_Depth_20252.png"
img = io.imread(file_path)

## Start from here.

In [1]:
# import os
# os.chdir('/Users/yebi/Library/CloudStorage/OneDrive-VirginiaTech/Research/Codes/research/BCS/BodyWeight') 

# import argparse
# parser = argparse.ArgumentParser(description = 'Extracting image descriptors from image')
# parser.add_argument('day', help = 'day info.')

# args = parser.parse_args()


import sys
from scipy import stats # summarize data

import os
import csv
from scipy.spatial import distance as dist
# from imutils import perspective
import numpy as np
import cv2
import pandas as pd
import predict_mrcnn
import skimage.io as io


In [ ]:
# rootdir = "./DairyLMC"
rootdir = "/Volumes/MyPassport1"
# depthdir = "./DairyLMC/D1/depth/4973AM"
temp_dep = "/Volumes/MyPassport1/" + "D1" + "/depth/"
temp_csv = "/Volumes/MyPassport1/" + "D1" + "/CSV/"
# csvdir = "./DairyLMC/D1/CSV/4973AM/"
temp_day = "/Users/yebi/Library/CloudStorage/OneDrive-VirginiaTech/Research/Codes/research/BCS/BodyWeight/outputs/mrcnn/" + "D1" + "/" + "D1" + "_"
i = 1

for cowid in os.listdir(temp_dep):
      summ = os.path.join(temp_day+cowid+".csv")
  # if os.path.isfile(summ):
  #   print("already there, go to next one")
  #   continue
  # else:
      depthdir = temp_dep + cowid + "/"
      csvdir = temp_csv + cowid + "/"
      with open(summ, "w", newline = "") as output:
          writer = csv.writer(output)
          writer.writerow(["Day", "ID", "Frame", "Width", "Length", "Height_Centroid", "Height_average", "Volume"])
          for root, dirs, files in os.walk(depthdir):
            Day = root.split("/")[3]
            ID = root.split("/")[5]

            for j in np.arange(3, len(files), 15):
                file = files[j]
                file_path = root + file
                # file_path= "/Volumes/MyPassport1/D1/depth/5724PM/_Depth_7639.png"
                if file_path == "/Volumes/MyPassport1/D1/depth/4973AM/.__Depth_1237.png":
                    continue

                print("Now is running: ", file_path)
                
                # Build na of summary file.
                frame = os.path.splitext(file)[0]
                width = np.nan
                length = np.nan
                height0 = np.nan 
                height1 = np.nan 
                volume = np.nan

                # Read in images.
                img = io.imread(file_path)

                # Read in distance csv files
                filename = os.path.splitext(file)[0]+".csv"
                # print(filename)
                csv_path = os.path.join(csvdir, filename)
                dfcsv = pd.read_csv(csv_path, header = None) #read in depth csv file

                # Using mrcnn to predict contour (result is binary image with size 848*480) 
                fill_img = predict_mrcnn.predict_mrcnn(img) #bianry image after mrcnn 
                
                #mrcnn can just give us overall mask, but we need the contour, so we have to draw it again.
                cnts, _ = cv2.findContours(fill_img.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE) 
                cmax = max(cnts, key=cv2.contourArea) 

                  ##part1: calculate width and length
                rect = cv2.minAreaRect(cmax)
                box = cv2.boxPoints(rect)
                (A, B, C, D) = np.int0(box)
                d0 = dist.euclidean(B, C)
                d1 = dist.euclidean(A, B)
                width = min(d0, d1)
                length = max(d0, d1)

                  ##part2: calculated height: centroid method
                M = cv2.moments(cmax)
                row_centroid = int(M["m01"] / M["m00"])
                col_centroid  = int(M["m10"] / M["m00"])
                height0 = 2.94 - dfcsv.iloc[row_centroid , col_centroid]

                  ##part3: calculate height: average method
                pixel = np.argwhere(fill_img == 255) #find pixels for white part
                dfcsv_rows = [] #combine pixel and distance 
                for row, col in pixel:
                  dfcsv_rows.append([row, col, dfcsv.iloc[row, col]])
                df = pd.DataFrame(dfcsv_rows, columns = ['row', 'col', 'dist'])

                df.dist.replace(to_replace=0, value = df.dist.mean(), inplace=True) #replace 0 with average distance
                height1 = 2.94 - df.dist.mean()

                  ##part4: calculate volume
                df["height"] = 2.94 - df["dist"] #build new column named height
                volume = sum(df.height)

                  ##part5: write into csv file.
                frame = os.path.splitext(file)[0]
                writer.writerow([Day, ID, frame, width, length, height0, height1, volume])
                i = i + 1
                print("####################### Done %d ############################" %i)   